## SQL

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
# the information about parameters of connection to the database is hidden due to education platform prescription

In [4]:
books = pd.io.sql.read_sql('select * from books', con = engine)
books.info()
books.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


In [5]:
authors = pd.io.sql.read_sql('select * from authors', con = engine)
authors.info()
authors.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


In [6]:
publishers = pd.io.sql.read_sql('select * from publishers', con = engine)
publishers.info()
publishers.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


In [7]:
ratings = pd.io.sql.read_sql('select * from ratings', con = engine)
ratings.info()
ratings.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


In [8]:
reviews = pd.io.sql.read_sql('select * from reviews', con = engine)
reviews.info()
reviews.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


#### *Calculate the number of reviews and average rating for each book*

In [9]:
query_1 = """SELECT 
                COUNT(*) AS cnt 
             FROM books 
             WHERE publication_date > '2000-01-01'
          """

In [10]:
pd.io.sql.read_sql(query_1, con = engine)

,cnt
0,819


#### *Calculate the number of reviews and average rating for each book*

In [11]:
query_2 = """SELECT 
                RATINGS.book_id AS book_id,
                BOOKS.title AS book_title,
                   AVG(RATINGS.rating) AS avg_rating,
                   COUNT(REVIEWS.text) AS count_reviews
            FROM RATINGS
            LEFT JOIN REVIEWS ON 
                      REVIEWS.book_id = RATINGS.book_id AND
                      REVIEWS.username = RATINGS.username
            LEFT JOIN BOOKS ON 
                      BOOKS.book_id = RATINGS.book_id                           
            GROUP BY RATINGS.book_id, BOOKS.title 
            ORDER BY count_reviews DESC
          """

In [12]:
data_query_2 = pd.io.sql.read_sql(query_2, con = engine)
data_query_2.head(10)

,book_id,book_title,avg_rating,count_reviews
0,948,Twilight (Twilight #1),3.662500,7
1,497,Outlander (Outlander #1),4.125000,6
2,963,Water for Elephants,3.977273,6
3,299,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
4,207,Eat Pray Love,3.395833,6
5,734,The Glass Castle,4.206897,6
6,779,The Lightning Thief (Percy Jackson and the Oly...,4.080645,6
7,627,The Alchemist,3.789474,6
8,854,The Road,3.772727,6
9,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6


#### *Identify the publisher with the most books thicker than 50 pages*

In [13]:
query_3 = """SELECT 
                PUBLISHERS.publisher AS publisher,
                    COUNT(BOOKS.book_id) AS count_books
            FROM PUBLISHERS
            LEFT JOIN BOOKS ON 
                      BOOKS.publisher_id = PUBLISHERS.publisher_id
            WHERE BOOKS.num_pages > '50'
            GROUP BY PUBLISHERS.publisher
            ORDER BY count_books DESC
            LIMIT 5
          """

In [14]:
pd.io.sql.read_sql(query_3, con = engine)

,publisher,count_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


#### *Identify the author with the highest average book rating - count only books with 50 or more ratings*

In [15]:
query_4_1 = """
                 SELECT 
                     authors.author AS author,
                     AVG(top1_rating.avg_rating) as top_avg_rating
                 FROM
                        (SELECT  new_books.book_id,
                            books.author_id AS author_id,
                            AVG(ratings.rating) AS avg_rating
                        FROM
                                (SELECT
                                    ratings.book_id,
                                    COUNT(ratings.rating) as count_rating
                                FROM ratings                
                                GROUP BY book_id            
                                HAVING COUNT(ratings.rating) >= '50'
                                ) as new_books
                        LEFT JOIN books 
                               ON books.book_id = new_books.book_id
                        LEFT JOIN ratings 
                               ON ratings.book_id = new_books.book_id
                        GROUP BY new_books.book_id, books.author_id
                        ORDER BY avg_rating DESC
                        ) as top1_rating
                 LEFT JOIN authors 
                        ON authors.author_id = top1_rating.author_id 
                 GROUP BY author
                 ORDER BY top_avg_rating DESC                           
             """

In [16]:
pd.io.sql.read_sql(query_4_1, con = engine)

,author,top_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


In [17]:
query_4_2 = """
                 SELECT
                     book_author_ratings.author_id AS author_id,
                     authors.author AS author,
                     AVG(book_author_ratings.ratings) AS avg_ratings
                 FROM
                       (SELECT 
                            new_books.book_id AS book_id,
                            books.author_id AS author_id,
                            ratings.rating AS ratings                           
                       FROM
                                (SELECT
                                    ratings.book_id,
                                    COUNT(ratings.rating) as count_rating
                                FROM ratings                
                                GROUP BY book_id            
                                HAVING COUNT(ratings.rating) >= '50') as new_books
                      LEFT JOIN books 
                             ON books.book_id = new_books.book_id
                      LEFT JOIN ratings 
                             ON ratings.book_id = new_books.book_id) AS book_author_ratings
                 LEFT JOIN authors
                        ON authors.author_id = book_author_ratings.author_id  
             
                GROUP BY book_author_ratings.author_id, authors.author 
                ORDER BY avg_ratings DESC
                                       
            """

##### Alternative option

In [18]:
pd.io.sql.read_sql(query_4_2, con = engine)

,author_id,author,avg_ratings
0,236,J.K. Rowling/Mary GrandPré,4.287097
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.246914
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645
5,621,William Golding,3.901408
6,235,J.D. Salinger,3.825581
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,630,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,372,Lois Lowry,3.750000


#### *Calculate the average number of reviews from users who gave more than 50 ratings*

In [19]:
query_5 = """SELECT avg(count_reviews)
                FROM
                    (
                    SELECT 
                        COUNT(reviews.text) AS count_reviews
                    FROM
                        ratings
                    LEFT JOIN reviews ON reviews.book_id = ratings.book_id
                                     AND reviews.username = ratings.username
                    GROUP BY ratings.username
                    HAVING COUNT(ratings.rating) > '50'
                    ) as counts
            """

In [20]:
pd.io.sql.read_sql(query_5, con = engine)

,avg
0,24.333333
